In [1]:
import pandas as pd
import boto3
import io
import datetime

# Step 1

## Load patient_id_month_year

In [5]:
bucket_name = 'waymark-assignment'
object_key = 'patient_id_month_year.csv'
aws_access_key_id = AWS_ACCESS_KEY_ID
aws_secret_access_key = AWS_SECRET_ACCESS_KEY

s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

# Fetch the file from the S3 bucket
obj = s3.get_object(Bucket=bucket_name, Key=object_key)
file_content = obj['Body'].read().decode('utf-8')

# Load the CSV content into a DataFrame
patient_id_month_year = pd.read_csv(io.StringIO(file_content))

/var/folders/v8/s4lwxw8s6wxbkhy0gqkmypnc0000gn/T/ipykernel_61053/568008719.py:17: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  patient_id_month_year = pd.read_csv(io.StringIO(file_content))


In [3]:
patient_id_month_year

,patient_id,month_year,Unnamed: 2
0,ID0001,1/1/23,NaN
1,ID0001,2/1/23,NaN
2,ID0001,3/1/23,NaN
3,ID0001,6/1/23,NaN
4,ID0001,8/1/23,NaN
...,...,...,...
1047121,NaN,NaN,NaN
1047122,NaN,NaN,NaN
1047123,NaN,NaN,NaN
1047124,NaN,NaN,NaN


## Clean data

In [4]:
patient_id_month_year = patient_id_month_year[['patient_id','month_year']]
patient_id_month_year = patient_id_month_year[~patient_id_month_year.patient_id.isna()] #remove all NULL patient_id rows
patient_id_month_year['month_year'] = pd.to_datetime(patient_id_month_year['month_year'], format='%m/%d/%y')

## Transform month_year into enrollment periods

In [5]:
to_append = [] #use a list to append enrollment periods for each patient then concatenate at the end for all patients

for patient_id in patient_id_month_year.patient_id.unique():

    #sort by dates just in case
    df = patient_id_month_year[patient_id_month_year.patient_id==patient_id].sort_values(by='month_year',ascending=True)

    
    # find the difference in days between the i and i+1 rows
    df['diff'] = df['month_year'].diff().dt.days
    
    # a gap is defined to be >31 day difference (i.e. more than a month)
    df['gap'] = df['diff'] > 31

    # assign a group number for each enrollment period
    df['group'] = df['gap'].cumsum()

    # group by each "group" to get the first and last month of each enrollment period
    continuous_enrollments = df.groupby('group').agg(enrollment_start_date=('month_year', 'first'),
                                                     enrollment_end_date=('month_year', 'last')
                                                    ).reset_index(drop=True)
    
    continuous_enrollments['patient_id'] =  patient_id

    # offset to get the month end date for enrollment
    continuous_enrollments['enrollment_end_date'] = continuous_enrollments['enrollment_end_date'] + pd.offsets.MonthEnd(0)

    to_append.append(continuous_enrollments[['patient_id','enrollment_start_date','enrollment_end_date']])
    
patient_enrollment_plan = pd.concat(to_append)
patient_enrollment_plan.reset_index(drop=True, inplace=True)

In [6]:
patient_enrollment_plan.head()

,patient_id,enrollment_start_date,enrollment_end_date
0,ID0001,2023-01-01,2023-03-31
1,ID0001,2023-06-01,2023-06-30
2,ID0001,2023-08-01,2023-08-31
3,ID0001,2023-11-01,2023-12-31
4,ID0002,2023-02-01,2023-04-30


## Answer 1: Report the number of rows in patient_enrollment_span.csv

In [7]:
len(patient_enrollment_plan)

3105

# Step 2: Data Aggregation

## Load outpatient_visits

In [8]:
bucket_name = 'waymark-assignment'
object_key = 'outpatient_visits_file.csv'
aws_access_key_id = AWS_ACCESS_KEY_ID
aws_secret_access_key = AWS_SECRET_ACCESS_KEY

s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

# Fetch the file from the S3 bucket
obj = s3.get_object(Bucket=bucket_name, Key=object_key)
file_content = obj['Body'].read().decode('utf-8')

# Load the CSV content into a DataFrame
outpatient_visits= pd.read_csv(io.StringIO(file_content))

/var/folders/v8/s4lwxw8s6wxbkhy0gqkmypnc0000gn/T/ipykernel_59174/4288309250.py:17: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  outpatient_visits= pd.read_csv(io.StringIO(file_content))


In [9]:
outpatient_visits

,patient_id,date,outpatient_visit_count,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,ID0001,1/3/23,2.0,NaN,NaN,NaN,NaN,NaN
1,ID0001,1/8/23,2.0,NaN,NaN,NaN,NaN,NaN
2,ID0001,1/9/23,2.0,NaN,NaN,NaN,NaN,NaN
3,ID0001,1/15/23,2.0,NaN,NaN,NaN,NaN,NaN
4,ID0001,1/21/23,4.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1043900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1043901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1043902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1043903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Clean data

In [10]:
outpatient_visits = outpatient_visits[['patient_id','date','outpatient_visit_count']]
outpatient_visits = outpatient_visits[~outpatient_visits.patient_id.isna()] #remove all NULL patient_id rows
outpatient_visits['date'] = pd.to_datetime(outpatient_visits['date'], format='%m/%d/%y')

## Merge patient_enrollment_plan with outpatient_visits

In [11]:
result = patient_enrollment_plan.merge(outpatient_visits,how='left',on='patient_id')

# turn NULL from outpatient_visit_count into 0, but keeping the NULL for date (for counting the number of days with visits later)
result['outpatient_visit_count'] = result['outpatient_visit_count'].fillna(0)
result['outpatient_visit_count'] = result['outpatient_visit_count'].astype(int)

In [12]:
# keep only dates that falls within their designated enrollment period, and NULL dates 

result = result[((result.date>=result.enrollment_start_date) & (result.date<=result.enrollment_end_date)) | result.date.isna()]

In [13]:
result.head()

,patient_id,enrollment_start_date,enrollment_end_date,date,outpatient_visit_count
0,ID0001,2023-01-01,2023-03-31,2023-01-03,2
1,ID0001,2023-01-01,2023-03-31,2023-01-08,2
2,ID0001,2023-01-01,2023-03-31,2023-01-09,2
3,ID0001,2023-01-01,2023-03-31,2023-01-15,2
4,ID0001,2023-01-01,2023-03-31,2023-01-21,4


## Aggregate

In [14]:
# Sum the visit counts in the same enrollment period, and count unique days with visit in the same enrollment period (for each patient)

result = result.groupby(['patient_id','enrollment_start_date','enrollment_end_date']).agg(ct_outpatient_visits=('outpatient_visit_count','sum'),
                                                                                        ct_days_with_outpatient_visit=('date','nunique'),
                                                                                       ).reset_index()


In [15]:
result.head()

,patient_id,enrollment_start_date,enrollment_end_date,ct_outpatient_visits,ct_days_with_outpatient_visit
0,ID0001,2023-01-01,2023-03-31,17,8
1,ID0001,2023-06-01,2023-06-30,6,2
2,ID0001,2023-08-01,2023-08-31,6,2
3,ID0001,2023-11-01,2023-12-31,5,3
4,ID0002,2023-02-01,2023-04-30,19,7


## Answer 2: Report the number of distinct values of ct_days_with_outpatient_visit in result.csv

In [16]:
result['ct_days_with_outpatient_visit'].nunique()

33